# K-Fold 交叉验证 (Cross-Validation)

在机器学习建模过程中，通行的做法通常是将数据分为训练集和测试集。测试集是与训练独立的数据，完全不参与训练，用于最终模型的评估。在训练过程中，经常会出现过拟合的问题，就是模型可以很好的匹配训练数据，却不能很好在预测训练集外的数据。如果此时就使用测试数据来调整模型参数，就相当于在训练时已知部分测试数据的信息，会影响最终评估结果的准确性。通常的做法是在训练数据再中分出一部分做为验证(Validation)数据，用来评估模型的训练效果。

验证数据取自训练数据，但不参与训练，这样可以相对客观的评估模型对于训练集之外数据的匹配程度。模型在验证数据中的评估常用的是交叉验证，又称循环验证它，将原始数据分成K组(K-Fold)，将每个子集数据分别做一次验证集，其余的K-1组子集数据作为训练集，这样会得到K个模型。这K个模型分别在验证集中评估结果，最后的误差MSE(Mean Squared Error)加分平均就得到交叉验证误差。K一般大于等于2，实际操作时一般从3开始取，只有在原始数据集合数据量小的时候才会尝试取2。交叉验证有效利用了有限的数据，并且评估结果能够尽可能接近模型在测试集上的表现，可以做为模型优化的指标使用。

下面举一个具体的例子来说明K-Fold的过程，比如有如下的数据

```
[0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
```

分为K=3组后

```
Fold1: [0.5, 0.2]
Fold2: [0.1, 0.3]
Fold3: [0.4, 0.6]
```

交叉验证的时会使用如下三个模型，分别进行训练和测试，每个测试集误差MSE加和平均就得到了交叉验证的总评分

```
Model1: Trained on Fold1 + Fold2, Tested on Fold3
Model2: Trained on Fold2 + Fold3, Tested on Fold1
Model3: Trained on Fold1 + Fold3, Tested on Fold2
```

## K-Fold Sklearn API

Scikit中指供了K-Fold API，n-split就是K值，shuffle指是否对数据洗牌，random_state=1随机种子

In [3]:
# scikit-learn k-fold cross-validation
from numpy import array
from sklearn.model_selection import KFold
# data sample
data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
# prepare cross validation
kfold = KFold(n_splits=3, shuffle = True, random_state= 1)
# enumerate splits
for train, test in kfold.split(data):
	print('train: %s, test: %s' % (data[train], data[test]))

train: [0.1 0.4 0.5 0.6], test: [0.2 0.3]
train: [0.2 0.3 0.4 0.6], test: [0.1 0.5]
train: [0.1 0.2 0.3 0.5], test: [0.4 0.6]


## K-Fold in Cross Validation

Scikit中提取带K-Fold接口的交叉验证接口sklearn.model_selection.cross_validate，但是该接口没有数据shuffle功能，所以一般结合Kfold一起使用。如果Train数据在分组前已经经过了shuffle处理，比如使用`train_test_split`分组，那就可以直接使用`cross_val_score`接口

In [4]:
import numpy as np
from sklearn import svm, datasets
from sklearn.model_selection import cross_val_score

iris = datasets.load_iris()
X, y = iris.data, iris.target

clf = svm.SVC(kernel='linear', C=1, random_state=0)

n_folds = 5
kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X)
scores = cross_val_score(clf, X, y, scoring='precision_macro', cv = kf)

scores[0]

0.9696969696969697

K值的选取会影响bias和viriance。K越大，每次投入的训练集的数据越多，模型的Bias越小。但是K越大，又意味着每一次选取的训练集之前的相关性越大，而这种大相关性会导致最终的test error具有更大的Variance。一般来说，根据经验我们一般选择k=5或10。